# Time series clustering exercise

# Import Packages

In [1]:
import os
import pandas as pd
import numpy as np
import zipfile 
import geopandas as gpd
import pylab as pl
from scipy.cluster.vq import kmeans2,kmeans, whiten
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist, pdist
import sklearn.cluster
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from IPython.display import display, clear_output
import time
% pylab inline
import scipy.cluster as scp

Populating the interactive namespace from numpy and matplotlib


# Data

Download the census bureau business data for all years 1993-2014. You can investigate using the API (I have not done it with the census bureau). I did is as you see below

In [ ]:
###############################################################################
# Download all datasets
#these commands can be run on the shell and get the data with the command wget
#the cell needs to be run only once
!for ((y=93; y<=99; y+=1)); do wget \
ftp://ftp.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp$y\totals.zip; done

!for ((y=0; y<=1; y+=1)); do wget \
ftp://ftp.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp0$y\totals.zip; done

!for ((y=2; y<=9; y+=1)); do wget \
ftp://ftp.census.gov/econ200$y\/CBP_CSV/zbp0$y\totals.zip; done

!for ((y=10; y<=14; y+=1)); do wget \
ftp://ftp.census.gov/econ20$y\/CBP_CSV/zbp$y\totals.zip; done



--2017-12-19 10:34:50--  ftp://ftp.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp93totals.zip
           => “zbp93totals.zip”
Resolving ftp.census.gov... 148.129.75.35, 2610:20:2010:a09:1000:0:9481:4b23
Connecting to ftp.census.gov|148.129.75.35|:21... connected.
Logging in as anonymous ... 
Error in server response, closing control connection.
Retrying.

--2017-12-19 10:39:51--  ftp://ftp.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp93totals.zip
  (try: 2) => “zbp93totals.zip”
Connecting to ftp.census.gov|148.129.75.35|:21... connected.
Logging in as anonymous ... 

In [ ]:
###############################################################################
# Move data to folder in PUIDATA
os.system('mkdir ' + os.getenv('PUIDATA') + '/NYC_ZIPBusinessData')
for y in range(93,100,1):
    os.system('mv zbp' + str(y) + 'totals.zip ' + os.getenv('PUIDATA') + '/NYC_ZIPBusinessData')

for y in range(0,10,1):
    os.system('mv zbp0' + str(y) + 'totals.zip ' + os.getenv('PUIDATA') + '/NYC_ZIPBusinessData')
    
for y in range(10,15,1):
    os.system('mv zbp' + str(y) + 'totals.zip ' + os.getenv('PUIDATA') + '/NYC_ZIPBusinessData')

 Download the NYC zipcodes shapefile. One of many ways in which you can get the zipcodes shapefile for NYC
 https://data.cityofnewyork.us/download/i8iw-xf4u/application%2Fzip


In [ ]:
###############################################################################
# Download and unzip Zipcode shapefiles into PUIdata
!wget https://data.cityofnewyork.us/download/i8iw-xf4u/application%2Fzip -O NYCZipcodeShapes.zip
os.system('mkdir ' + os.getenv('PUIDATA') + '/NYC_Zipcode_Shapefile')
os.system("unzip " + 'NYCZipcodeShapes.zip' + " -d " + os.getenv('PUIDATA') + '/NYC_Zipcode_Shapefile')
os.system('mv NYCZipcodeShapes.zip ' + os.getenv('PUIDATA') + '/NYC_Zipcode_Shapefile' )

In [ ]:
###############################################################################
ZipShape = gpd.read_file(os.getenv('PUIDATA') +\
                         '/NYC_Zipcode_Shapefile/ZIP_CODE_040114.shp')

In [ ]:
###############################################################################
# Only take relevant info from Zip shape file
ZipShapedf = ZipShape[['ZIPCODE','geometry']]

# Format columns into desired format for merging later on
ZipShapedf['ZIPCODE'] = pd.to_numeric(ZipShapedf['ZIPCODE'], errors='coerce')
ZipShapedf.rename(columns={'ZIPCODE':'zip'},inplace=True)

# Review data
ZipShapedf.head()

## You can use zipfile module in python to unzip the files
it should be install in your system, but if it is not you can get the code with wget from here
https://github.com/python/cpython/blob/2.7/Lib/zipfile.py
remembering to use the raw link
(or you can use the usual shell commands, and miss the chance to learn something new)

In [ ]:
###############################################################################
Businessdf = pd.DataFrame(columns={'zip','name','empflag','emp','qp1','ap',
                                   'est','year'})
Businessdf.head()

# For 94 to 99
for i in range(94,100,1):
    # Read in data using zip module
    fname = 'zbp' + ('{:02d}'.format(i)) + 'totals.zip'
    zf = zipfile.ZipFile(os.getenv('PUIDATA') + '/NYC_ZIPBusinessData' +\
                         '/' + fname)
    df = pd.read_csv(zf.open(fname.replace('.zip','.txt')))
    
    # The format and naming convetnions of the data changes, so to 
    # ensure consistency these two lines of code are used
    df.columns = map(str.lower, df.columns)
    df = df[['zip','name','empflag','emp','qp1','ap','est']]
    
    # Create column for year
    df['year'] = str(19) + ('{:02d}'.format(i))
    
    # Merge onto big dataframe
    Businessdf = pd.concat([Businessdf,df])

# For 2000 to 2014
for i in range(0,15,1):
    # Read in data using zip module
    fname = 'zbp' + ('{:02d}'.format(i)) + 'totals.zip'
    zf = zipfile.ZipFile(os.getenv('PUIDATA') + '/NYC_ZIPBusinessData' +\
                         '/' + fname)
    df = pd.read_csv(zf.open(fname.replace('.zip','.txt')))
    
    # The format and naming convetnions of the data changes, 
    # so to ensure consistence these two lines of code are used
    df.columns = map(str.lower, df.columns)
    df = df[['zip','name','empflag','emp','qp1','ap','est']]
    
    # Create column for year
    df['year'] = str(20) + ('{:02d}'.format(i))
    
    # Merge onto big dataframe
    Businessdf = pd.concat([Businessdf,df])

In [ ]:
Businessdf.zip = Businessdf.zip.astype(int)
Businessdf.head()

# Data cleaning

you may need to clean your data: for some NYC zip codes there may be no info
sanity check: you should have 20 (N_timestamps) datapoints per time series and about 250 zipcodes (Nzipcodes)


IMPORTANT: we talked about the importance of "whitening" your data.
Whitenings decorrelates the data: it makes the features independent so that the data covariance matrix is the identity matrix.
Whitening your data in time series analysis is in most cases **wrong**: you are modifying your time behaviour. This is because of the strong correlation between features (two consecutive time stamps for the same observation, the same zip code here, are strongly correlated). Here instead you want to standardize your time series: subtract the mean and divide each time series (separately) by its standard deviation. As a sanity check (if you use skitlearn Kmeans or skitlearns kmeans2): you want your data array to be shaped Nzipcodes x Ntimestamps

mydata.shape should be (Nzipcodes, Ntimestamps)

mydata[i].std() shoould be 1 for all i in range(len(Nzipcodes))

mydata[i].mean() should be ~0 for all i in range(len(Nzipcodes))



## TASKS:
    
    1. get and prep your data.
    2. cluster the NUMBER OF ESTABLISHMENTS time series with K-means in **a few** clusters (as discussed there is no real good, sound way to decide what a good number is here. try a few options, keeping in mind a few is more than a couple, but i recommand you stay within the single digit numbers)
    3. plot the cluster centers (if you used K means those are the means of the clusters). you can plot for example the cluster centers overlayed on each time series (using the alpha channel to control the opacity in the plot may be helpful here).
    4. Use another clustering algorithm (of your choice)
    5. overlay your data on a NYC map: you can use shapefiles for the zip codes and different colors for different clusters
    6. Compare the results of the 2 algorithms
    7. attempt an interpretation. this is dangerous ground: clustering is an exploratory tool so you do not want to jump to conclusions because you see some clusters! but seeing structure in your data can inform your next moves as an investigator. 
    

# Get and Prep the Data

In [ ]:
# Merge df with Zip Shape file to ensure only NYC zips are included
Businessdf_ = pd.merge(Businessdf,ZipShapedf,on='zip').reset_index()

In [ ]:
# Check to see differences in data sets between years
print('year','Number of Rows','Unique Zips')
for i in range(1994,2015,1):
    print(i,len(Businessdf_.zip[Businessdf_.year == str(i)]),
          len(Businessdf_.zip[Businessdf_.year == str(i)].unique()))


It appears that there are multiple years with null data sets for differing zips, however, once we remove duplicates these disappear. It appears to be an issue with the polygons.

In [ ]:
###############################################################################
# Drop duplicate data sets - seem to be identical except for geometry
Businessdf_.drop_duplicates(['zip','year'],inplace=True)

# Pivot table to pivot zips against years
Businessdf = Businessdf_.pivot(columns='zip', index='year', values='est')


In [ ]:
Businessdf.head()

In [ ]:
# Create a normal dataframe to normalise
NormBusinessdf = Businessdf.copy()

# Create loop that normalises row by row
for i in list(NormBusinessdf.columns):
    rowmean = NormBusinessdf[i].mean()
    rowstd = NormBusinessdf[i].std()
    NormBusinessdf[i] = (NormBusinessdf[i] - rowmean) / rowstd
    
# Review data 
NormBusinessdf.head()

In [ ]:
# Check for Null Values
sum(NormBusinessdf.isnull().sum())

In [ ]:
NormBusinessdf.shape

In [ ]:
# Drop all values that are Null
NormBusinessdf.dropna(axis=1,inplace=True)

In [ ]:
NormBusinessdf.shape
# We lose 15 Rows of Data

# Cluster the number of Establishments

### First plot the time series for each zipcode

In [ ]:
# Set up figure
pl.figure(figsize=(12,6))

# Plot Figure
pl.plot(NormBusinessdf,alpha=0.1,c='k');

# Format Plot
pl.title('Time series of Number of \nestablishments per Zipcode',
         fontsize=20)
pl.xlabel('year', fontsize=15)
pl.ylabel('Normalised Number of Establishments',fontsize=15)

#### Figure 1: Time series of Normalised Number of Establishments per year from 1994 to 2014
It is important to note that there is a clear trend that most of the zipcodes follow, which is indicated by the darker middle section, however there are multiple zipcodes that do not follow the same trend as can be seen by all the 'noisy' offshoot lines

# Use Elbow test to determine correct number of Clusters

In [ ]:
###############################################################################
# Function courtesy of Prof Sobolevsky
def elbow(data,K):
#data is your input as numpy form
#K is a list of number of clusters you would like to show.
    # Run the KMeans model and save all the results for each number of clusters
    KM = [KMeans(n_clusters=k).fit(data) for k in K]
    
    # Save the centroids for each model with a increasing k
    centroids = [k.cluster_centers_ for k in KM]

    # For each k, get the distance between the data with each center. 
    D_k = [cdist(data, cent, 'euclidean') for cent in centroids]
    
    # But we only need the distance to the nearest centroid since 
    # we only calculate dist(x,ci) for its own cluster.
    globals()['dist'] = [np.min(D,axis=1) for D in D_k]
    
    # Calculate the Average SSE.
    avgWithinSS = [sum(d)/data.shape[0] for d in dist]
    
    
    # elbow curve
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(K, avgWithinSS, 'b*-')
    plt.grid(True)
    plt.xlabel('Number of clusters')
    plt.ylabel('Average within-cluster sum of squares')
    plt.title('Elbow for KMeans clustering')
    plt.show()
    
    
    # Total with-in sum of square plot. Another way to show the result.
    wcss = [sum(d**2) for d in dist]
    tss = sum(pdist(data)**2)/data.shape[0]
    bss = tss-wcss
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(K, bss/tss*100, 'b*-')
    plt.grid(True)
    plt.xlabel('Number of clusters')
    plt.ylabel('Percentage of variance explained')
    plt.title('Elbow for KMeans clustering')
    plt.show()

In [ ]:
elbow(NormBusinessdf.T, range(1,30))

#### Figure 2: Elbow Test output of different Cluster sizes
Note how there is no clear elbow, however the best fit appears to be between 4 and 6, where the increase in cluster size starts to provide less percentage explanation in variance.

# Create Kmeans Clustering

In [ ]:
# Create empty arrays
clusters = []
centers = []

# Run 3 types of clusters
for n in range(4,7,1):
    km = KMeans(random_state=300, n_clusters=n)
    res = km.fit(NormBusinessdf.T)
    clusters.append(res.labels_)
    centers.append(res.cluster_centers_)

In [ ]:
# Create transposed dataframe
NormBusinessdfT = NormBusinessdf.T.copy()

# Add clusters onto Dataframe
NormBusinessdfT['4 Clusters'] = clusters[0]
NormBusinessdfT['5 Clusters'] = clusters[1]
NormBusinessdfT['6 Clusters'] = clusters[2]

In [ ]:
###############################################################################
# Define Colour Scheme
colours = ['r','g','b','orange','purple','k']

for i in range(4,7,1):
    # Set up plots
    fig, ax = pl.subplots(figsize=(12,6))
    for p in range(0,i,1):
        # Plot Time Series with colours and centroids
        pl.plot(NormBusinessdfT[NormBusinessdfT[str(i) + \
                                                ' Clusters']==p].T[:-3],
                c=colours[p],alpha=0.075)
        pl.plot(list(NormBusinessdfT.columns)[:-3],centers[i-4][p],
                c=colours[p])
        
        # Format Plots
        pl.title(str(i) + ' Clusters - KMeans',fontsize=15)
        pl.xlabel('year', fontsize=15)
        pl.ylabel('Normalised Number of Establishments',fontsize=15)
        pl.ylim(-3,4)
        
        # For Animation to display
        time.sleep(0.01) # too slow it down
        clear_output(wait=True)
        display(fig)



#### Figure 3: The Clustered time series data plotted with 3 different clusters (4,5,6)
Note how the solid line indicating the kmean value per year moves through the center of the lighter lines as expected. It is important to note that although in each case there is one clearly defined cluster, the others tend to have more variability as the noisier time series tend to to not follow as much of a pattern. 


Visually it looks like the 5 clusters may be the best in this case, as the lines seem to follow a specific trend with little deviation, whereas it becomes messy at 6 clusters and appears too coarse at 4 clusters.

# Other means of clustering - Using Dendograms

In [ ]:
###############################################################################
fig = pl.figure(figsize=(20,60))

ddg = scp.hierarchy.dendrogram(scp.hierarchy.linkage(NormBusinessdf.T, 
                                                     method='ward'),
                               labels = list(NormBusinessdf.columns),
                               leaf_rotation=270.,leaf_font_size=20, 
                               truncate_mode='lastp', p=250, 
                               show_contracted=True, color_threshold = 14, 
                               orientation='left', above_threshold_color='k'
                              )  # font size for the x axis labels)
#pl.xticks(rotation=90)
pl.xlabel("Distance", fontsize=20)
pl.ylabel("ZipCodes", fontsize=20)
pl.yticks(fontsize=15, rotation=0)
pl.xticks(fontsize=15)
pl.grid('off')

#### Figure 4: Heirarchical Clustering Displayed in a Dendogram for time series of number of establishments by different zipcodes 
Note how we have set the threshold to 14 and we have 5 distinct clusters in this scenario. However, this threshold can be adjusted to show different grouoings and different levels.

This also follows the Kmeans clustering idication that there is a big cluster that most zipcodes follow (sjown as the teal group in the above figure).

# Other Clustering Methods -  Agglomerative

In [ ]:
# Create empty arrays
clustersagc = []

# Run 3 types of clusters
for n in range(4,7,1):
    agc = AgglomerativeClustering(n_clusters=n,  
                                  compute_full_tree=True).fit(NormBusinessdf.T)
    clustersagc.append(agc.labels_)

In [ ]:
# Add clusters onto Dataframe
NormBusinessdfT['4 Clusters AGC'] = clustersagc[0]
NormBusinessdfT['5 Clusters AGC'] = clustersagc[1]
NormBusinessdfT['6 Clusters AGC'] = clustersagc[2]

In [ ]:
###############################################################################
# Define Colour Scheme
colours = ['r','g','b','orange','purple','k']


for i in range(4,7,1):
    # Set up plots
    fig, ax = pl.subplots(figsize=(12,6))
    for p in range(0,i,1):
        # Plot Time Series with colours and centroids
        pl.plot(NormBusinessdfT[NormBusinessdfT[str(i) + \
                                                ' Clusters AGC']==p].T[:-6],
                c=colours[p],alpha=0.075)
        pl.plot(list(NormBusinessdfT.columns)[:-6],
                NormBusinessdfT[NormBusinessdfT[str(i) + ' Clusters AGC'\
                                               ] ==p].mean()[:-6],c=colours[p])
        
        # Format Plots
        pl.title(str(i) + ' Clusters - Agglomerative - Ward Linkage',
                 fontsize=15)
        pl.xlabel('year', fontsize=15)
        pl.ylabel('Normalised Number of Establishments',fontsize=15)
        pl.ylim(-3,4)
        
        # For Animation to display
        time.sleep(0.01) # too slow it down
        clear_output(wait=True)
        display(fig)

#### Figure 5: Agglomeratuve - Ward Linkage Clustering with 4, 5 and 6 clusters

Note how the clusters appear very similar to the Kmeans method, although slightly different in some cases. There are similar things to note with the one strong cluster and the other clusters with more noisy data.

In [ ]:
# Reviww clusters to compare differences
NormBusinessdfT[['4 Clusters','4 Clusters AGC',
                 '5 Clusters', '5 Clusters AGC', 
                 '6 Clusters','6 Clusters AGC']]

The clusters are mostly identical with different numbering schemes, however there are some differences between the two tyes of clustering methods.

# Now to plot the data on maps

First we need to merge the shapefile onto the data set.

In [ ]:
# Merge data with polygons and turn into Geopandas data frame
Mapdf = pd.merge(NormBusinessdfT.reset_index(),ZipShapedf,on='zip')
Mapdf = gpd.GeoDataFrame(Mapdf)

### Now plot

In [ ]:
###############################################################################
# Functions to inout discrete colour bars
# Found at http://sensitivecities.com/so-youd-like-to-make-a-
# map-using-python-EN.html#.WjKrVN-nFPY
def colorbar_index(ncolors, cmap, labels=None, **kwargs):
    """
    This is a convenience function to stop you making off-by-one errors
    Takes a standard colour ramp, and discretizes it,
    then draws a colour bar with correctly aligned labels
    """
    cmap = cmap_discretize(cmap, ncolors)
    mappable = cm.ScalarMappable(cmap=cmap)
    mappable.set_array([])
    mappable.set_clim(-0.5, ncolors+0.5)
    colorbar = plt.colorbar(mappable, **kwargs)
    colorbar.set_ticks(np.linspace(0, ncolors, ncolors))
    colorbar.set_ticklabels(range(ncolors))
    if labels:
        colorbar.set_ticklabels(labels)
    return colorbar

def cmap_discretize(cmap, N):
    """
    Return a discrete colormap from the continuous colormap cmap.

        cmap: colormap instance, eg. cm.jet. 
        N: number of colors.

    Example
        x = resize(arange(100), (5,100))
        djet = cmap_discretize(cm.jet, 5)
        imshow(x, cmap=djet)

    """
    if type(cmap) == str:
        cmap = get_cmap(cmap)
    colors_i = np.concatenate((np.linspace(0, 1., N), (0., 0., 0., 0.)))
    colors_rgba = cmap(colors_i)
    indices = np.linspace(0, 1., N + 1)
    cdict = {}
    for ki, key in enumerate(('red', 'green', 'blue')):
        cdict[key] = [(indices[i], colors_rgba[i - 1, ki], 
                       colors_rgba[i, ki]) for i in xrange(N + 1)]
    return matplotlib.colors.LinearSegmentedColormap(cmap.name + "_%d" % N, 
                                                     cdict, 1024)

In [ ]:
# Plot for each of the 3 clusters selected for Kmeans
for i in range(4,7,1):

    # Set up figures
    fig,ax = plt.subplots(figsize=(15,10))

    # Plot Data
    Mapdf.plot(column = str(i) + ' Clusters',ax=ax, 
               edgecolor='black', lw=0.5, alpha=1, 
               cmap='Blues_r',legend=False)

    # Format Plot
    ax.set_title(str(i) + ' Clusters - KMeans',fontsize=15)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.axis('off');
    
    # Add a colour bar
    cb = colorbar_index(ncolors=i, cmap='Blues', labels=list(np.arange(1,i+1,1)))
    cb.ax.tick_params(labelsize=15)
    
    # For Animation to display
    time.sleep(1) # too slow it down
    clear_output(wait=True)
    display(fig)


#### Figure 6: Maps showing kmean clustering of the zipcodes by number of establishments per Zipcode
For the most part, most zipcodes behave the same, however we see the biggest difference in areas in Midetown and Downtown Manhattan, which makes sense as this areas has changed substantially over the time period, and is also home to many businesses. There are a couple other areas on the outskirts of the city that also fall into different clusters. This could be attributed to the fact that it is cheaper to get property on the outskirts and could have something to do with more industrial businesses setting up shop at different time periods.

In [ ]:
# Plot for each of the 3 clusters selected for Agglomorative
for i in range(4,7,1):

    # Set up figures
    fig,ax = plt.subplots(figsize=(15,10))

    # Plot Data
    Mapdf.plot(column = str(i) + ' Clusters AGC',ax=ax, 
               edgecolor='black', lw=0.5, alpha=1, 
               cmap='Blues_r',legend=False)

    # Format Plot
    ax.set_title(str(i) + ' Clusters - Agglomeative - Ward Linkage ',
                 fontsize=15)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.axis('off');
    
    # Add a colour bar
    cb = colorbar_index(ncolors=i, cmap='Blues', labels=list(np.arange(1,i+1,1)))
    cb.ax.tick_params(labelsize=15)
    
    # For Animation to display
    time.sleep(1) # too slow it down
    clear_output(wait=True)
    display(fig)


#### Figure 7: Maps showing agglomerative clustering of the zipcodes by number of establishments per Zipcode
For the most part, most zipcodes behave the same, however we see the biggest difference in areas in Midetown and Downtown Manhattan, which makes sense as this areas has changed substantially over the time period, and is also home to many businesses. There are a couple other areas on the outskirts of the city that also fall into different clusters. This could be attributed to the fact that it is cheaper to get property on the outskirts and could have something to do with more industrial businesses setting up shop at different time periods.

These maps are almost identical to the Kmean Clustering.

# Next Moves

Both methods of clustering showed similar results, we may want to compare the clustering with several other types of clustering methods to see if there are any major differences. For the most part, the results indicate what we would expect to see and that is Midtown and Downtown behavong dfferently to the rest of New York, most likely due to the number of businesses in these areas and because they are areas that have changed drastically in the last 20 years. It may be interesting to take the analysis slightly further and see if we can see the impacts spreading into downtown Brooklyn or potentially differences in Gentrified areas. However none of this is conclusive. 

We see similarly that the outskirts of the city behave differently. This could be attributed to the fact that these may be indistrial areas with small businesses set up to feed and support indistry as they have been pushed further from the city center due to densification and rising costs.

An important next step would be to understand which clusters are seeing businesses grow in number and which are decreasing, this could potentially be used by the city to identify areas where businesses are struggling to thrive and maybe understand what they can do support these businesses and ensure there is some form of job creation and economic stimulation in some of tehse areas and the city overall. It would be very important to try and investigate if the results can be confirmed with some groundtruth and facts about the areas that are clustered together. 

## ASH Score 100